In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import yaml 
import pandas as pd
from sklearn.metrics import accuracy_score
import xgboost as xgb
import pickle
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import RidgeClassifierCV

%matplotlib inline


Using TensorFlow backend.


In [5]:
def dataset_dt_splt(df, st_dt, ed_st):
    return df[(df.date> st_dt) & (df.date <= ed_st)]
def utility(dates: np.ndarray, returns: np.ndarray,
            weights: np.ndarray, actions: np.ndarray) -> np.ndarray:
    gains = pd.DataFrame({
        'date': dates,
        'gain': weights * returns * actions
    })

    p = gains.groupby('date').sum().to_numpy()

    i = len(set(dates))

    t = p.sum() / np.sqrt(np.power(p, 2).sum()) * np.sqrt(250 / i)

    return np.clip(t, a_min=0, a_max=6) * p.sum()

In [6]:
train = pd.read_csv('input/train.csv').astype('float32')
# train = pd.read_csv('/content/drive/MyDrive/janeStreetKaggle/input/train.csv').astype('float32')
## TODO not sure if it's the right thing to do
## very unbalance if include 0 weight, maybe use imblearn.RandomSampler
train = train[train['weight'] != 0]
train['action'] = (train.weight*train.resp>0).astype('int')


In [8]:
## max weight
max_weight = train.weight.max()
max_dt = train.date.max()
feat_cols = [col for col in train.columns if 'feature' in col] + ['weight']
## First Change: Feeding Sample Weight in clf.fit
# train_weight = train['weight']

datasets = {}
splt_dates = [0, 250, 375, 500]

ds_train = dataset_dt_splt(train, splt_dates[0], splt_dates[1])
ds_val = dataset_dt_splt(train, splt_dates[1], splt_dates[2])


print("MinMaxScaler")
sc = MinMaxScaler().fit(ds_train[feat_cols])
scale_clean_X = sc.transform(ds_train[feat_cols])

# print("StandardScaler")
# sc = StandardScaler().fit(ds_train[feat_cols])
# scale_clean_X = sc.transform(ds_train[feat_cols])



print("MeanImputing")

si = SimpleImputer(strategy='mean').fit(scale_clean_X)
scale_clean_X = si.transform(scale_clean_X)

y_train = ds_train.action


gc.collect()


MinMaxScaler
MeanImputing


20

In [ ]:
for i, feat in enumerate(feat_cols):

    print(feat)
    g = sns.displot(x=ds_train[feat], hue=y_train)
    g.fig.set_size_inches(20,20)
    # plt.show()
    

    g.savefig("pairplot.png", dpi=800)    
    plt.close()

feature_0
feature_1
feature_2
feature_3
feature_4
feature_5
feature_6
feature_7
feature_8
feature_9
feature_10
feature_11
feature_12
feature_13
feature_14
feature_15
feature_16
feature_17
feature_18
feature_19
feature_20


/Users/kingf.wong/.conda/envs/FinanceML/lib/python3.7/site-packages/seaborn/axisgrid.py:392: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(nrow, ncol, **kwargs)


feature_21


/Users/kingf.wong/.conda/envs/FinanceML/lib/python3.7/site-packages/seaborn/axisgrid.py:392: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(nrow, ncol, **kwargs)


feature_22


...

/Users/kingf.wong/.conda/envs/FinanceML/lib/python3.7/site-packages/seaborn/axisgrid.py:392: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(nrow, ncol, **kwargs)


feature_108


/Users/kingf.wong/.conda/envs/FinanceML/lib/python3.7/site-packages/seaborn/axisgrid.py:392: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(nrow, ncol, **kwargs)


feature_109


/Users/kingf.wong/.conda/envs/FinanceML/lib/python3.7/site-packages/seaborn/axisgrid.py:392: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(nrow, ncol, **kwargs)
